In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
%cd C:\Users\asche\deep_learning\exam-project\kunskapskontroll_2

C:\Users\asche\deep_learning\exam-project\kunskapskontroll_2


In [ ]:
from flask import Flask, render_template, request, Response, jsonify
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import base64

app = Flask(__name__)

# Load the pre-trained models and labels
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
emotion_model = load_model('model.h5', compile=False)

def preprocess_image(image_data):
    # Convert the data URL to an OpenCV image
    encoded_data = image_data.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)  # Assuming color image

    # Convert image to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Rescale pixel values to be between 0 and 1
    gray_image = gray_image.astype('float32') / 255.0

    # Resize the image to match the model's expected input shape
    gray_image = cv2.resize(gray_image, (48, 48))
    gray_image = np.expand_dims(gray_image, axis=0)  # Add batch dimension
    gray_image = np.expand_dims(gray_image, axis=-1)  # Add channel dimension

    return gray_image

def predict_emotion_from_image(image_data):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_data)

    # Predict the emotion using your model
    emotion_prediction = emotion_model.predict(preprocessed_image)
    predicted_emotion = emotion_labels[np.argmax(emotion_prediction)]

    return predicted_emotion

def generate_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform face detection
        faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Extract the face region
            face = gray[y:y + h, x:x + w]

            # Preprocess the face for emotion prediction
            preprocessed_face = cv2.resize(face, (48, 48))
            preprocessed_face = preprocessed_face / 255.0
            preprocessed_face = preprocessed_face.reshape(1, 48, 48, 1)

            # Predict the emotion
            predicted_emotion = predict_emotion_from_image(frame)  

        # Display the frame with annotations
        cv2.imshow('Emotion Detector', frame)
        cv2.waitKey(500)  # Delay of 100 milliseconds (0.1 second) between frames

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')


@app.route('/predict_emotion', methods=['POST'])
def predict_emotion():
    # Extract image data from the POST request
    image_data = request.json.get('image_data')

    # Preprocess the image (you'll need to implement this based on your model's requirements)

    # Predict the emotion using your model
    predicted_emotion = predict_emotion_from_image(image_data)  # You'll need to implement this

    # Return the predicted emotion
    return jsonify({'emotion': predicted_emotion})


if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
